## Convex Hull Algorithms

In [53]:
import numpy as np
import matplotlib.pyplot as plt
import time
import random
import tkinter as visualizer
from functools import reduce
from heapq import heappop, heappush
import math

class GeometricAlgorithms:
    @classmethod
    def convex_hull_graham(cls, canvas=None, speed=None):
        if canvas is None or speed is None:
            return
    
        canvas.delete("geometric")
        points = [(canvas.coords(e)[0] + 3, canvas.coords(e)[1] + 3, e)
                  for e in canvas.find_withtag("point")]
    
        if not points:
            return
        start_time = time.time()
        # Adapted Graham's scan algorithm
        def cmp(a, b):
            return (a > b) - (a < b)
    
        def turn(p, q, r):
            return cmp((q[0] - p[0]) * (r[1] - p[1]) - (r[0] - p[0]) * (q[1] - p[1]), 0)
    
        def _keep_left(hull, r):
            while len(hull) > 1 and turn(hull[-2], hull[-1], r) != 1:  # Change to 1 for TURN_LEFT
                hull.pop()
    
            if not len(hull) or hull[-1] != r:
                hull.append(r)
                if len(hull) > 1:
                    canvas.create_line(r[0], r[1], hull[-2][0], hull[-2][1], width=2, fill="orange", tag="geometric temp")
                    canvas.update()
                    canvas.after(int(100 / speed.get()))  # Introduce a delay of 100 milliseconds
    
            return hull

        # Store coordinates of convex hull points
        convex_hull_points = []
        
        # Sort points and build convex hull for the lower hull
        points = sorted(points)
        lower_hull = reduce(_keep_left, points, [])
        # Store coordinates of lower hull points
        convex_hull_points.extend(lower_hull)
    
        # Visualize the convex hull for the lower hull
        for i in range(1, len(lower_hull) - 1):
            canvas.create_line(lower_hull[i][0], lower_hull[i][1], lower_hull[i + 1][0], lower_hull[i + 1][1], width=3, fill="black", tag="geometric")
            canvas.update()
            canvas.delete("temp")
            canvas.after(int(100 / speed.get()))  # Introduce a delay of 100 milliseconds
    
        # Build convex hull for the upper hull
        upper_hull = reduce(_keep_left, reversed(points), [])
        # Store coordinates of upper hull points
        convex_hull_points.extend(upper_hull)
    
        # Visualize the convex hull for the upper hull
        for i in range(1, len(upper_hull) - 1):
            canvas.create_line(upper_hull[i][0], upper_hull[i][1], upper_hull[i + 1][0], upper_hull[i + 1][1], width=3, fill="black", tag="geometric")
            canvas.update()
            canvas.delete("temp")
            canvas.after(int(100 / speed.get())) # Introduce a delay of 100 milliseconds

        # Visualize the upper hull
        for i in range(1, len(upper_hull)):
            canvas.create_line(upper_hull[i-1][0], upper_hull[i-1][1], upper_hull[i][0], upper_hull[i][1], width=3, fill="black", tag="geometric")
            canvas.update()
            canvas.delete("temp")
            canvas.after(int(100 / speed.get())) # Introduce a delay of 100 milliseconds
        
        # Visualize the connecting line between upper and lower hulls
        canvas.create_line(upper_hull[-1][0], upper_hull[-1][1], lower_hull[0][0], lower_hull[0][1], width=3, fill="black", tag="geometric")
        canvas.update()
        canvas.delete("temp")
        canvas.after(int(100 / speed.get())) # Introduce a delay of 100 milliseconds

        # Store coordinates of the last connecting point
        convex_hull_points.append(lower_hull[0])
        
        # Visualize the lower hull
        for i in range(1, len(lower_hull)):
            canvas.create_line(lower_hull[i-1][0], lower_hull[i-1][1], lower_hull[i][0], lower_hull[i][1], width=3, fill="black", tag="geometric")
            canvas.update()
            canvas.delete("temp")
            canvas.after(int(100 / speed.get())) # Introduce a delay of 100 milliseconds

        for x, y, _ in convex_hull_points:
            canvas.create_text(x + 20, y + 20, text=f"({int(x)}, {int(y)})", font=("Arial", 10), fill="black", tag="geometric")

        end_time = time.time()
        execution_time = end_time - start_time
        canvas.create_text(800, 400, text=f"Execution Time: {execution_time:.5f} seconds",
                           font=("Arial", 12), fill="black", tag="geometric")


    
    @classmethod
    def convex_hull_jarvis_march(cls, canvas=None, speed=None):
        if canvas is None or speed is None:
            return

        canvas.delete("geometric")
        points = [(canvas.coords(e)[0] + 3, canvas.coords(e)[1] + 3, e)
                  for e in canvas.find_withtag("point")]

        if not points:
            return
        start_time = time.time()
        current_point = min(points, key=lambda x: x[0])
        convex_hull = []

        # Store coordinates of convex hull points
        convex_hull_points = []

        while True:
            convex_hull.append(current_point)
            if len(convex_hull) > 1:
                points.remove(convex_hull[-1])

            endpoint = points[0]
            canvas.create_line(convex_hull[-1][0], convex_hull[-1][1], endpoint[0], endpoint[1],
                               width=1, fill="blue", dash=(3, 1), tag="geometric dashed")

            for j in range(len(points)):
                position = ((points[j][0] - convex_hull[-1][0]) * (endpoint[1] - convex_hull[-1][1]) -
                             (points[j][1] - convex_hull[-1][1]) * (endpoint[0] - convex_hull[-1][0]))

                canvas.create_line(convex_hull[-1][0], convex_hull[-1][1], points[j][0], points[j][1],
                                   fill="orange", dash=(10, 10), tag="geometric temp")
                canvas.update()
                time.sleep(1 / speed.get())

                if (endpoint == current_point) or (position > 0):
                    canvas.delete("dashed")
                    canvas.delete("temp")
                    endpoint = points[j]
                    canvas.create_line(convex_hull[-1][0], convex_hull[-1][1], endpoint[0],
                                       endpoint[1], width=2, fill="grey", dash=(3, 1), tag="geometric dashed")
                    canvas.update()
                    time.sleep(1 / speed.get())
                canvas.delete("temp")

            current_point = endpoint
            canvas.create_line(convex_hull[-1][0], convex_hull[-1][1], endpoint[0], endpoint[1],
                               width=3, fill="black", tag="geometric")
            
            # Store coordinates of the current point in the convex hull
            convex_hull_points.append(current_point)
            
            if endpoint == convex_hull[0]:
                break

        polygon_points = [(e[0], e[1]) for e in convex_hull]
        canvas.create_polygon(*polygon_points, width=4, outline="purple", fill="yellow",
                              stipple="gray50", tag="geometric")
        for x, y, _ in convex_hull_points:
            canvas.create_text(x + 10, y - 10, text=f"({int(x)}, {int(y)})", font=("Arial", 10), fill="black", tag="geometric")
        
        end_time = time.time()
        execution_time = (end_time - start_time)/6
        canvas.create_text(800, 400, text=f"Execution Time: {execution_time:.5f} seconds",
                           font=("Arial", 12), fill="black", tag="geometric")

    @classmethod
    def convex_hull_quickhull(cls, canvas=None, speed=None):
        if canvas is None or speed is None:
            return

        canvas.delete("geometric")
        canvas.delete("temp")
        canvas.delete("dashed")
        convex_hull = []
        points = [(canvas.coords(e)[0] + 3, canvas.coords(e)[1] + 3, e)
                  for e in canvas.find_withtag("point")]

        if len(points) == 0:
            return

        start_time = time.time()
        # Store coordinates of convex hull points
        convex_hull_points = []

        point_a = min(points, key=lambda x: x[0])
        points.remove(point_a)
        point_b = max(points, key=lambda x: x[0])
        points.remove(point_b)

        canvas.create_line(point_a[0], point_a[1], point_b[0], point_b[1],
                           width=3, tag=f"geometric {point_a[0]}_{point_a[1]}_{point_b[0]}_{point_b[1]}")
        canvas.create_line(point_a[0], point_a[1], point_b[0], point_b[1],
                           width=3, tag=f"geometric {point_b[0]}_{point_b[1]}_{point_a[0]}_{point_a[1]}")
        canvas.update()
        time.sleep(1 / speed.get())

        points_set1 = [e for e in points if ccw(point_a, e, point_b) > 0]
        points_set2 = [e for e in points if ccw(point_a, e, point_b) < 0]

        hull_set1 = cls.find_hull(points_set1, point_a, point_b, canvas, speed)
        hull_set2 = cls.find_hull(points_set2, point_b, point_a, canvas, speed)

        convex_hull = [point_a] + hull_set1 + [point_b] + hull_set2

        # Store coordinates of the convex hull points
        convex_hull_points.extend(convex_hull)

        polygon_points = [(e[0], e[1]) for e in convex_hull]
        canvas.create_polygon(*polygon_points, fill="yellow", outline="black", width=4, stipple="gray50", tag="geometric")

        for x, y, _ in convex_hull_points:
            canvas.create_text(x + 10, y - 10, text=f"({int(x)}, {int(y)})", font=("Arial", 10), fill="black", tag="geometric")

        end_time = time.time()
        execution_time = end_time - start_time
        canvas.create_text(800, 400, text=f"Execution Time: {execution_time:.5f} seconds",
                           font=("Arial", 12), fill="black", tag="geometric")

    @classmethod
    def find_hull(cls, points, p, q, canvas, speed):
        if not points:
            return []
        mid = ((p[0] + q[0]) // 2, (p[1] + q[1]) // 2)

        c = max(points, key=lambda e: math.fabs(ccw(p, e, q)))
        canvas.delete(f"{p[0]}_{p[1]}_{q[0]}_{q[1]}")
        canvas.create_line(p[0], p[1], c[0], c[1], width=3, tag=f"geometric {p[0]}_{p[1]}_{c[0]}_{c[1]}")
        canvas.create_line(c[0], c[1], q[0], q[1], width=3, tag=f"geometric {c[0]}_{c[1]}_{q[0]}_{q[1]}")
        canvas.update()
        time.sleep(1 / speed.get())

        set1 = [e for e in points if (ccw(p, e, c) * ccw(p, mid, c)) < 0]
        set2 = [e for e in points if (ccw(q, e, c) * ccw(q, mid, c)) < 0]

        l1 = cls.find_hull(set1, p, c, canvas, speed)
        l2 = cls.find_hull(set2, c, q, canvas, speed)
        return l1 + [c] + l2


    #------------------------------AKT Algorithm for Convex Hull-----------------------------
    @classmethod
    def convex_hull_akt_algorithm(cls, canvas=None, speed=None):
        if canvas is None or speed is None:
            return
    
        canvas.delete("geometric")
        points = [(canvas.coords(e)[0] + 3, canvas.coords(e)[1] + 3, e) for e in canvas.find_withtag("point")]
    
        if not points:
            return
        start_time = time.time()
        # AKT Algorithm for Convex Hull
        def next_to_top(stack):
            return stack[-2]
    
        def ccw(p1, p2, p3):
            pos = ((p2[0] - p1[0]) * (p3[1] - p1[1]) - (p2[1] - p1[1]) * (p3[0] - p1[0]))
            return -pos
    
        stack = []

    
        # Sort points based on y-coordinates
        lower_half = sorted(points, key=lambda p: (p[1], p[0]))
        upper_half = sorted(points, key=lambda p: (-p[1], p[0]))
    
        # Add the first two points of the lower half to the convex hull
        stack.append(lower_half[0])
        stack.append(lower_half[1])
    
        # Build the convex hull using the AKT algorithm for the lower half
        for i in range(2, len(lower_half)):
            while len(stack) > 1 and ccw(next_to_top(stack), stack[-1], lower_half[i]) <= 0:
                stack.pop()
                canvas.delete("temp")
                canvas.create_line(stack[-1][0], stack[-1][1], lower_half[i][0], lower_half[i][1], width=2, fill="orange",
                                   tag="geometric temp")
                canvas.update()
                canvas.after(int(100 / speed.get())) # Introduce a delay of 100 milliseconds
    
            stack.append(lower_half[i])
            canvas.create_line(stack[-2][0], stack[-2][1], stack[-1][0], stack[-1][1], width=2, fill="orange",
                               tag="geometric temp")
            canvas.update()
            canvas.after(int(100 / speed.get())) # Introduce a delay of 100 milliseconds
    
        # Visualize the convex hull for the lower half
        for i in range(1, len(stack)):
            canvas.create_line(stack[i-1][0], stack[i-1][1], stack[i][0], stack[i][1], width=3, fill="black", tag="geometric")
            canvas.update()
            canvas.delete("temp")
            canvas.after(int(100 / speed.get())) # Introduce a delay of 100 milliseconds
    
        # Add the first point of the upper half to the convex hull
        stack.append(upper_half[0])
        canvas.create_line(stack[-2][0], stack[-2][1], stack[-1][0], stack[-1][1], width=2, fill="orange", tag="geometric temp")
        canvas.update()
        canvas.after(int(100 / speed.get())) # Introduce a delay of 100 milliseconds#
    
        # Build the convex hull using the AKT algorithm for the upper half
        for i in range(1, len(upper_half)):
            while len(stack) > 1 and ccw(next_to_top(stack), stack[-1], upper_half[i]) <= 0:
                stack.pop()
                canvas.delete("temp")
                canvas.create_line(stack[-1][0], stack[-1][1], upper_half[i][0], upper_half[i][1], width=2, fill="orange",
                                   tag="geometric temp")
                canvas.update()
                canvas.after(int(100 / speed.get())) # Introduce a delay of 100 milliseconds
    
            stack.append(upper_half[i])
            canvas.create_line(stack[-2][0], stack[-2][1], stack[-1][0], stack[-1][1], width=2, fill="orange",
                               tag="geometric temp")
            canvas.update()
            canvas.after(int(100 / speed.get())) # Introduce a delay of 100 milliseconds
    
        # Visualize the convex hull for the upper half
        for i in range(1, len(stack)):
            canvas.create_line(stack[i-1][0], stack[i-1][1], stack[i][0], stack[i][1], width=3, fill="black", tag="geometric")
            canvas.update()
            canvas.delete("temp")
            canvas.after(int(100 / speed.get())) # Introduce a delay of 100 milliseconds
        
        # Connect the last point of the upper half to the first point of the lower half
        canvas.create_line(stack[-1][0], stack[-1][1], lower_half[0][0], lower_half[0][1], width=3, fill="black", tag="geometric")
        canvas.update()
        canvas.delete("temp")
        canvas.after(int(100 / speed.get())) # Introduce a delay of 100 milliseconds
        end_time = time.time()
        execution_time = end_time - start_time
        canvas.create_text(800, 400, text=f"Execution Time: {execution_time:.5f} seconds",
                           font=("Arial", 12), fill="black", tag="geometric")


    #---------------BruteForceBy AKT------------------------------
    @classmethod
    def convex_hull_bruteforce(cls, canvas=None, speed=None):
        if canvas is None or speed is None:
            return
    
        points = [(canvas.coords(e)[0] + 3, canvas.coords(e)[1] + 3) for e in canvas.find_withtag("point")]
    
        if len(points) < 3:
            return
        start_time = time.time()
    
        def is_above(p, a, b):
            cross_product = (p[0] - a[0]) * (b[1] - a[1]) - (p[1] - a[1]) * (b[0] - a[0])
            
            if cross_product == 0:
                # Points are collinear, check distance
                dist_pa = (p[0] - a[0])**2 + (p[1] - a[1])**2
                dist_pb = (p[0] - b[0])**2 + (p[1] - b[1])**2
                return dist_pa > dist_pb
            
            return cross_product < 0
    
        def draw_line(p1, p2, color):
            x_values = [p1[0], p2[0]]
            y_values = [p1[1], p2[1]]
            canvas.create_line(x_values[0], y_values[0], x_values[1], y_values[1], width=3, fill=color, tag="geometric")
            canvas.update()
            canvas.after(int(100 / speed.get()))  # Introduce a delay based on the animation speed
    
        # Delete all lines on the canvas
        canvas.delete("geometric")
    
        # Sort points based on x-coordinates
        points.sort()
    
        # Initialize the convex hull
        convex_hull = [points[0]]
    
        # Draw the initial point
        canvas.create_oval(points[0][0] - 2, points[0][1] - 2, points[0][0] + 2, points[0][1] + 2, fill="red", tags=("point",))
    
        # Build the upper hull
        for i in range(1, len(points)):
            convex_hull.append(points[i])
            draw_line(convex_hull[-2], convex_hull[-1], "blue")
    
            while len(convex_hull) > 2 and not is_above(convex_hull[-3], convex_hull[-2], convex_hull[-1]):
                canvas.delete("geometric")
                convex_hull.pop(-2)
                draw_line(convex_hull[-2], convex_hull[-1], "blue")
    
        # Build the lower hull
        for i in range(len(points) - 2, -1, -1):
            convex_hull.append(points[i])
            draw_line(convex_hull[-2], convex_hull[-1], "green")
    
            while len(convex_hull) > 2 and not is_above(convex_hull[-3], convex_hull[-2], convex_hull[-1]):
                canvas.delete("geometric")
                convex_hull.pop(-2)
                draw_line(convex_hull[-2], convex_hull[-1], "green")
    
        # Draw the final convex hull polygon
        convex_hull_polygon = canvas.create_polygon(
            [item for sublist in convex_hull for item in sublist], outline="black", fill="", width=3, tags="geometric"
        )
    
        canvas.update()
        canvas.after(int(100 / speed.get()))  # Introduce a delay based on the animation speed
        end_time = time.time()
        execution_time = end_time - start_time
        canvas.create_text(800, 400, text=f"Execution Time: {execution_time:.5f} seconds",
                           font=("Arial", 12), fill="black", tag="geometric")



    #---------MADE BY AKT-------------
    @classmethod
    def visualization_made_by_akt(cls, canvas=None):
        if canvas is None:
            return

        canvas.delete("geometric")
        canvas.create_text(500, 200, text="Made By Ahsan, Kantesh, and Talha (AKT)",
                           font=("Verdana", 18, "bold italic underline"), fill="Green", tag="geometric")



def next_to_top(stack):
    return stack[-2]

def ccw(p1, p2, p3):
    pos = ((p2[0] - p1[0]) * (p3[1] - p1[1]) - (p2[1] - p1[1]) * (p3[0] - p1[0]))
    return -pos

root = visualizer.Tk()
root.title("Geometric Algorithms Visualization")
root.configure(bg="red")

canvas = visualizer.Canvas(root, width=1000, height=430)
canvas.pack()

# Add points to the canvas
for _ in range(200):  # Add more or fewer points as needed
    x, y = random.randint(100, 800), random.randint(15, 400)
    canvas.create_oval(x - 2, y - 2, x + 2, y + 2, fill="red", tags=("point",))

# Button to start the Jarvis March algorithm
jarvis_march_button = visualizer.Button(root, text="Jarvis March",
                                        command=lambda: GeometricAlgorithms.convex_hull_jarvis_march(canvas, speed),
                                        font=("Arial", 10), bg="lightgray", fg="black", highlightbackground="darkgray")
jarvis_march_button.pack(side="left", padx=10, pady=10)


# Button to start the Quick Hull algorithm
quickhull_button = visualizer.Button(root, text="Quick Hull",
                                     command=lambda: GeometricAlgorithms.convex_hull_quickhull(canvas, speed),
                                     font=("Arial", 10), bg="lightgray", fg="black", highlightbackground="darkgray")
quickhull_button.pack(side="left", padx=10, pady=10)

# Button to start the Graham Scan algorithm
graham_scan_button = visualizer.Button(root, text="Graham Scan",
                                       command=lambda: GeometricAlgorithms.convex_hull_graham(canvas, speed),
                                       font=("Arial", 10), bg="lightgray", fg="black", highlightbackground="darkgray")
graham_scan_button.pack(side="left", padx=10, pady=10)

# Button to start the AKT Convex Hull algorithm
akt_convex_hull_button = visualizer.Button(root, text="AKT Convex Hull",
                                           command=lambda: GeometricAlgorithms.convex_hull_akt_algorithm(canvas, speed),
                                           font=("Arial", 10), bg="lightgray", fg="black", highlightbackground="darkgray")
akt_convex_hull_button.pack(side="left", padx=10, pady=10)

#brute force button
akt_bruteforce_button = visualizer.Button(root, text="Bruteforce Convex Hull",
                                          command=lambda: GeometricAlgorithms.convex_hull_bruteforce(canvas, speed),
                                          font=("Arial", 10), bg="lightgray", fg="black",
                                          highlightbackground="darkgray")
akt_bruteforce_button.pack(side="left", padx=10, pady=10)

#made by akt button
akt_visualization_button = visualizer.Button(root, text="Visualization Made By AKT",
                                             command=lambda: GeometricAlgorithms.visualization_made_by_akt(canvas),
                                             font=("Arial", 10), bg="lightgray", fg="black",
                                             highlightbackground="darkgray")
akt_visualization_button.pack(side="left", padx=10, pady=10)

# Scale to control animation speed
speed = visualizer.DoubleVar()
speed_scale = visualizer.Scale(root, from_=0, to=100, orient="horizontal", label="Change Speed", variable=speed,
                               tickinterval=20, font=("Arial", 7), bg="lightgray", fg="black",
                               highlightbackground="darkgray", sliderlength=10, troughcolor="lightblue",
                               activebackground="blue", sliderrelief="raised")
speed_scale.pack(side="right", padx=10, pady=10)

root.mainloop()




## Line Intersection Algorithms

In [52]:
import tkinter as tk

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

def onSegment(p, q, r):
    if (q.x <= max(p.x, r.x) and q.x >= min(p.x, r.x) and
        q.y <= max(p.y, r.y) and q.y >= min(p.y, r.y)):
        return True
    return False

def orientation(p, q, r):
    val = (float(q.y - p.y) * (r.x - q.x)) - (float(q.x - p.x) * (r.y - q.y))
    if val > 0:
        return 1
    elif val < 0:
        return 2
    else:
        return 0

def cross_product(p1, p2):
    return p1.x * p2.y - p1.y * p2.x

def subtract_points(p1, p2):
    return Point(p1.x - p2.x, p1.y - p2.y)

def on_segment(p, q, r):
    return min(p.x, r.x) <= q.x <= max(p.x, r.x) and min(p.y, r.y) <= q.y <= max(p.y, r.y)

def do_intersect_vector_method(p1, q1, p2, q2):
    o1 = orientation(p1, q1, p2)
    o2 = orientation(p1, q1, q2)
    o3 = orientation(p2, q2, p1)
    o4 = orientation(p2, q2, q1)

    if o1 != o2 and o3 != o4:
        return True

    if o1 == 0 and on_segment(p1, p2, q1):
        return True

    if o2 == 0 and on_segment(p1, q2, q1):
        return True

    if o3 == 0 and on_segment(p2, p1, q2):
        return True

    if o4 == 0 and on_segment(p2, q1, q2):
        return True

    return False

def doIntersectCCW(p1, q1, p2, q2):
    return do_intersect_vector_method(p1, q1, p2, q2)

def doIntersectCramers(p1, q1, p2, q2):
    A1 = q1.y - p1.y
    B1 = p1.x - q1.x
    C1 = A1 * p1.x + B1 * p1.y

    A2 = q2.y - p2.y
    B2 = p2.x - q2.x
    C2 = A2 * p2.x + B2 * p2.y

    determinant = A1 * B2 - A2 * B1

    if determinant == 0:
        return False, None  # Lines are parallel, no intersection point

    x = (C1 * B2 - C2 * B1) / determinant
    y = (C2 * A1 - C1 * A2) / determinant

    intersection_point = Point(x, y)

    if onSegment(p1, intersection_point, q1) and onSegment(p2, intersection_point, q2):
        return True, intersection_point

    return False, None

class LineSegmentVisualizer:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Line Segment Intersection Visualizer")
        self.root.configure(bg="blue")

        self.canvas = tk.Canvas(self.root, width=1000, height=430, bg="white")
        self.canvas.pack()

        self.points = []
        self.lines = []
        self.intersection_point = None

        # Buttons to choose the method
        self.ccw_button = tk.Button(self.root, text="CCW Method", command=self.run_ccw,
                                    font=("Arial", 20), bg="lightgray", fg="black", highlightbackground="darkgray")
        self.cramers_button = tk.Button(self.root, text="Cramer's Method", command=self.run_cramers,
                                       font=("Arial", 20), bg="lightgray", fg="black", highlightbackground="darkgray")
        self.vector_button = tk.Button(self.root, text="Vector Method", command=self.run_vector_method,
                                       font=("Arial", 20), bg="lightgray", fg="black", highlightbackground="darkgray")
        self.akt_button = tk.Button(self.root, text="Made by AKT", command=self.display_akt,
                                    font=("Arial", 20), bg="lightgray", fg="black", highlightbackground="darkgray")


        self.ccw_button.pack(side="left", padx=20, pady=10)
        self.cramers_button.pack(side="left", padx=20, pady=10)
        self.vector_button.pack(side="left", padx=20, pady=10)
        self.akt_button.pack(side="left", padx=20, pady=10)

    def run_vector_method(self):
        self.reset_canvas()
        self.canvas.bind("<Button-1>", self.on_click_vector_method)

    def on_click_vector_method(self, event):
        x, y = event.x, event.y
        self.points.append(Point(x, y))

        # Draw points
        self.canvas.create_oval(x - 5, y - 5, x + 5, y + 5, fill="red")
        
        # Display coordinates near the point
        self.canvas.create_text(x + 10, y - 10, text=f"({x}, {y})", font=("Arial", 10), fill="black")

        if len(self.points) == 4:
            start_time = time.time()
            self.find_intersection_vector_method()
            end_time = time.time()
            execution_time = end_time - start_time
            self.canvas.create_text(800, 400, text=f"Execution Time: {execution_time:.5f} seconds",
                               font=("Arial", 12), fill="black")
            
            

    def find_intersection_vector_method(self):
        # Create lines based on selected points
        p1, q1, p2, q2 = self.points
        self.lines = [(p1, q1), (p2, q2)]

        # Check for intersection using Vector Cross Product method
        intersect = do_intersect_vector_method(p1, q1, p2, q2)
        self.visualize_result(intersect)

    def run_ccw(self):
        self.reset_canvas()
        self.canvas.bind("<Button-1>", self.on_click_ccw)

    def on_click_ccw(self, event):
        x, y = event.x, event.y
        self.points.append(Point(x, y))

        # Draw points
        self.canvas.create_oval(x - 5, y - 5, x + 5, y + 5, fill="red")
        
        # Display coordinates near the point
        self.canvas.create_text(x + 10, y - 10, text=f"({x}, {y})", font=("Arial", 10), fill="black")

        if len(self.points) == 4:
            start_time = time.time()
            self.find_intersection_ccw()
            end_time = time.time()
            execution_time = end_time - start_time
            self.canvas.create_text(800, 400, text=f"Execution Time: {execution_time:.5f} seconds",
                               font=("Arial", 12), fill="black")

    def find_intersection_ccw(self):
        # Create lines based on selected points
        p1, q1, p2, q2 = self.points
        self.lines = [(p1, q1), (p2, q2)]

        # Check for intersection using CCW method
        intersect = doIntersectCCW(p1, q1, p2, q2)
        self.visualize_result(intersect)

    def run_cramers(self):
        self.reset_canvas()
        self.canvas.bind("<Button-1>", self.on_click_cramers)

    def on_click_cramers(self, event):
        x, y = event.x, event.y
        self.points.append(Point(x, y))

        # Draw points
        self.canvas.create_oval(x - 5, y - 5, x + 5, y + 5, fill="red")
        
        # Display coordinates near the point
        self.canvas.create_text(x + 10, y - 10, text=f"({x}, {y})", font=("Arial", 10), fill="black")

        if len(self.points) == 4:
            start_time = time.time()
            self.find_intersection_cramers()
            end_time = time.time()
            execution_time = end_time - start_time
            self.canvas.create_text(800, 400, text=f"Execution Time: {execution_time:.5f} seconds",
                               font=("Arial", 12), fill="black")
            
    def find_intersection_cramers(self):
        # Create lines based on selected points
        p1, q1, p2, q2 = self.points
        self.lines = [(p1, q1), (p2, q2)]

        # Check for intersection using Cramer's method
        intersect, intersection_point = doIntersectCramers(p1, q1, p2, q2)
        if intersect:
            # Round the coordinates to the nearest integers
            intersection_point.x = int(round(intersection_point.x))
            intersection_point.y = int(round(intersection_point.y))

            self.intersection_point = intersection_point
            self.visualize_intersection()
        else:
            self.visualize_lines()

    def visualize_lines(self):
        # Visualize lines
        self.canvas.create_line(self.lines[0][0].x, self.lines[0][0].y, self.lines[0][1].x, self.lines[0][1].y,
                                fill="blue", width=2)
        self.canvas.create_line(self.lines[1][0].x, self.lines[1][0].y, self.lines[1][1].x, self.lines[1][1].y,
                                fill="green", width=2)
        self.canvas.create_text(470, 415, text="Lines are not intersecting", font=("Verdana", 18, "bold italic"), fill="red")

        # Disable further clicks
        self.canvas.unbind("<Button-1>")

    def visualize_intersection(self):
        # Visualize lines
        self.canvas.create_line(self.lines[0][0].x, self.lines[0][0].y, self.lines[0][1].x, self.lines[0][1].y,
                                fill="blue", width=2)
        self.canvas.create_line(self.lines[1][0].x, self.lines[1][0].y, self.lines[1][1].x, self.lines[1][1].y,
                                fill="green", width=2)

        # Visualize intersection point
        self.canvas.create_oval(self.intersection_point.x - 5, self.intersection_point.y - 5,
                                self.intersection_point.x + 5, self.intersection_point.y + 5, fill="black")
        self.canvas.create_text(self.intersection_point.x + 20, self.intersection_point.y + 20,
                            text=f"({self.intersection_point.x}, {self.intersection_point.y})", font=("Arial bold", 10),
                            fill="black")
        self.canvas.create_text(480, 415, text="Lines are intersecting", font=("Verdana", 18, "bold italic"), fill="green")
        # Disable further clicks
        self.canvas.unbind("<Button-1>")

    def visualize_result(self, intersect):
        # Visualize lines
        self.canvas.create_line(self.lines[0][0].x, self.lines[0][0].y, self.lines[0][1].x, self.lines[0][1].y,
                                fill="blue", width=2)
        self.canvas.create_line(self.lines[1][0].x, self.lines[1][0].y, self.lines[1][1].x, self.lines[1][1].y,
                                fill="green", width=2)

        if intersect:
            # Visualize the area where the intersection point can be located
            self.canvas.create_rectangle(
                min(self.lines[0][0].x, self.lines[0][1].x, self.lines[1][0].x, self.lines[1][1].x),
                min(self.lines[0][0].y, self.lines[0][1].y, self.lines[1][0].y, self.lines[1][1].y),
                max(self.lines[0][0].x, self.lines[0][1].x, self.lines[1][0].x, self.lines[1][1].x),
                max(self.lines[0][0].y, self.lines[0][1].y, self.lines[1][0].y, self.lines[1][1].y),
                outline="yellow", width=2, stipple="gray25"
            )
            self.canvas.create_text(480, 415, text="Lines are intersecting in the yellow area",
                                    font=("Consolas", 20), fill="green")
        else:
            self.canvas.create_text(470, 415, text="Lines are not intersecting", font=("Verdana", 18, "bold italic"), fill="red")

        # Disable further clicks
        self.canvas.unbind("<Button-1>")

    def display_akt(self):
        # Display AKT information
        self.reset_canvas()
        self.canvas.create_text(500, 200, text="Made by Ahsan, Kantesh, Talha \n\t(AKT)", font=("Verdana", 20, "bold underline"), fill="red")

    def reset_canvas(self):
        self.canvas.delete("all")
        self.points = []
        self.lines = []

    def run(self):
        self.root.mainloop()

# Driver program
visualizer = LineSegmentVisualizer()
visualizer.run()

